In [1]:
#!py -m pip install selenium
#!py -m pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [3]:
# Funkcja do scrapowania danych o książce
def scrape_book_details(driver, url, xpaths):
    results = {}
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 1.5)

        # Akceptowanie ciasteczek
        try:
            accept_cookies_button = wait.until(
                EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", accept_cookies_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", accept_cookies_button)
        except Exception:
            pass

        # Klikanie "Pokaż więcej" (jeśli dostępne)
        try:
            show_more_button = wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[1]/div/button'))
            )
            if show_more_button.is_displayed() and show_more_button.is_enabled():
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more_button)
                time.sleep(1)
                show_more_button.click()
        except Exception:
            pass

        # Scrapowanie elementów
        for name, xpath in xpaths.items():
            try:
                if name == "Category":
                    elements = driver.find_elements(By.XPATH, xpath)
                    categories = [el.text for el in elements]
                    results[name] = ", ".join(categories) if categories else "NA"
                else:
                    element = driver.find_element(By.XPATH, xpath)
                    results[name] = element.text
            except Exception:
                results[name] = "NA"

            # Autor
            try:
                single_author_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[1]/div/div/span"
                multiple_authors_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/div[1]/div[2]/div/a/span"
                elements = driver.find_elements(By.XPATH, multiple_authors_xpath)
                if elements:
                    results["Author"] = ", ".join([el.text for el in elements])  # Łączy autorów w jeden ciąg
                else:
                    results["Author"] = driver.find_element(By.XPATH, single_author_xpath).text  # Pojedynczy autor
            except Exception:
                results["Author"] = "NA"

            # Lektor
            try:
                single_lector_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[2]/div/div/span"
                multiple_lectors_xpath = "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/div[2]/div[2]/div/a/span"
                elements = driver.find_elements(By.XPATH, multiple_lectors_xpath)
                if elements:
                    results["Lector"] = ", ".join([el.text for el in elements])  # Łączy lektorów w jeden ciąg
                else:
                    results["Lector"] = driver.find_element(By.XPATH, single_lector_xpath).text  # Pojedynczy lektor
            except Exception:
                results["Lector"] = "NA"

        # Scrapowanie daty publikacji
        try:
            publication_date = wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[1]/div/div/div/div[1]/p[2]'))
            ).text
            results["Publication_Date"] = publication_date
        except Exception:
            results["Publication_Date"] = "NA"

        # Scrapowanie informacji o tomie (serii)
        try:
            volume_info = driver.find_element(
                By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/a[3]/div/div/span'
            ).text
            results["Volume_Info"] = volume_info
        except Exception:
            results["Volume_Info"] = "Nie dotyczy"

    except Exception as e:
        print(f"Error scraping book details: {e}")
    return results

In [4]:
# Funkcja do scrapowania linków książek z podstrony
def get_book_links(driver, url):
    links = []
    try:
        driver.get(url)
        elements = driver.find_elements(By.CSS_SELECTOR, "a.typography_root__26lgA")
        for element in elements:
            href = element.get_attribute("href")
            if href and "https://www.bookbeat.com/pl/book/" in href:
                links.append(href)
    except Exception as e:
        print(f"Error scraping links: {e}")
    return links


In [5]:
# Główna pętla
base_url = "https://www.bookbeat.com/pl/search?page={}&query=Polski&Category=148"
all_books_data = []


In [6]:
# Definiowanie XPath-ów
xpaths = {
    "Title": "/html/body/div[1]/div[1]/div[2]/main/section[2]/div/div/div[1]/h1",
    "Book_Score": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/button/div/figure[1]/div/figcaption",
    "Number_of_ratings_book": "/html/body/div[1]/div[1]/div[2]/main/section[4]/div/div/div/div/div[2]/figure[1]/p",
    "Lector_Score": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/button/div/figure[2]/div/figcaption",
    "Number_of_ratings_lector": "/html/body/div[1]/div[1]/div[2]/main/section[4]/div/div/div/div/div[2]/figure[2]/p",
    "Duration": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[2]/span/div/span",
    "Category": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[1]/div/a",
    "Format": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[2]/div/a",
    "Language": "/html/body/div[1]/div[1]/div[2]/main/section[3]/div/div[3]/div[1]/div/div[3]/p[2]",
}

In [28]:
# Tworzenie przeglądarki tylko raz 1
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(1, 21):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 1.
Znaleziono 50 linków na stronie 2.
Znaleziono 50 linków na stronie 3.
Znaleziono 50 linków na stronie 4.
Znaleziono 50 linków na stronie 5.
Znaleziono 50 linków na stronie 6.
Znaleziono 50 linków na stronie 7.
Znaleziono 50 linków na stronie 8.
Znaleziono 50 linków na stronie 9.
Znaleziono 0 linków na stronie 10.
Znaleziono 50 linków na stronie 11.
Znaleziono 50 linków na stronie 12.
Znaleziono 0 linków na stronie 13.
Znaleziono 50 linków na stronie 14.
Znaleziono 50 linków na stronie 15.
Znaleziono 50 linków na stronie 16.
Znaleziono 50 linków na stronie 17.
Znaleziono 50 linków na stronie 18.
Znaleziono 0 linków na stronie 19.
Znaleziono 0 linków na stronie 20.


In [8]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book1_20.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book1_20.csv'")

Dane zostały zapisane do pliku 'book1_20.csv'


In [9]:
# Tworzenie przeglądarki tylko raz 2
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(21, 41):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 21.
Znaleziono 0 linków na stronie 22.
Znaleziono 50 linków na stronie 23.
Znaleziono 50 linków na stronie 24.
Znaleziono 0 linków na stronie 25.
Znaleziono 50 linków na stronie 26.
Znaleziono 50 linków na stronie 27.
Znaleziono 0 linków na stronie 28.
Znaleziono 0 linków na stronie 29.
Znaleziono 0 linków na stronie 30.
Znaleziono 50 linków na stronie 31.
Znaleziono 0 linków na stronie 32.
Znaleziono 50 linków na stronie 33.
Znaleziono 50 linków na stronie 34.
Znaleziono 50 linków na stronie 35.
Znaleziono 0 linków na stronie 36.
Znaleziono 0 linków na stronie 37.
Znaleziono 0 linków na stronie 38.
Znaleziono 50 linków na stronie 39.
Znaleziono 0 linków na stronie 40.


In [10]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book21_40.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book21_40.csv'")

Dane zostały zapisane do pliku 'book21_40.csv'


In [27]:
# Tworzenie przeglądarki tylko raz 3
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(41, 61):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 41.
Znaleziono 0 linków na stronie 42.
Znaleziono 0 linków na stronie 43.
Znaleziono 0 linków na stronie 44.
Znaleziono 50 linków na stronie 45.
Znaleziono 50 linków na stronie 46.
Znaleziono 50 linków na stronie 47.
Znaleziono 50 linków na stronie 48.
Znaleziono 0 linków na stronie 49.
Znaleziono 50 linków na stronie 50.
Znaleziono 50 linków na stronie 51.
Znaleziono 50 linków na stronie 52.
Znaleziono 50 linków na stronie 53.
Znaleziono 50 linków na stronie 54.
Znaleziono 50 linków na stronie 55.
Znaleziono 50 linków na stronie 56.
Znaleziono 50 linków na stronie 57.
Znaleziono 50 linków na stronie 58.
Znaleziono 0 linków na stronie 59.
Znaleziono 50 linków na stronie 60.


In [12]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book41_60.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book41_60.csv'")

Dane zostały zapisane do pliku 'book41_60.csv'


In [13]:
# Tworzenie przeglądarki tylko raz 4
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(61, 81):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 61.
Znaleziono 50 linków na stronie 62.
Znaleziono 0 linków na stronie 63.
Znaleziono 0 linków na stronie 64.
Znaleziono 0 linków na stronie 65.
Znaleziono 50 linków na stronie 66.
Znaleziono 50 linków na stronie 67.
Znaleziono 0 linków na stronie 68.
Znaleziono 0 linków na stronie 69.
Znaleziono 0 linków na stronie 70.
Znaleziono 50 linków na stronie 71.
Znaleziono 50 linków na stronie 72.
Znaleziono 50 linków na stronie 73.
Znaleziono 50 linków na stronie 74.
Znaleziono 50 linków na stronie 75.
Znaleziono 50 linków na stronie 76.
Znaleziono 50 linków na stronie 77.
Znaleziono 50 linków na stronie 78.
Znaleziono 50 linków na stronie 79.
Znaleziono 50 linków na stronie 80.


In [14]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book61_80.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book61_80.csv'")

Dane zostały zapisane do pliku 'book61_80.csv'


In [15]:
# Tworzenie przeglądarki tylko raz 5
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(81, 101):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 81.
Znaleziono 0 linków na stronie 82.
Znaleziono 0 linków na stronie 83.
Znaleziono 50 linków na stronie 84.
Znaleziono 50 linków na stronie 85.
Znaleziono 50 linków na stronie 86.
Znaleziono 50 linków na stronie 87.
Znaleziono 50 linków na stronie 88.
Znaleziono 0 linków na stronie 89.
Znaleziono 50 linków na stronie 90.
Znaleziono 50 linków na stronie 91.
Znaleziono 50 linków na stronie 92.
Znaleziono 50 linków na stronie 93.
Znaleziono 50 linków na stronie 94.
Znaleziono 50 linków na stronie 95.
Znaleziono 50 linków na stronie 96.
Znaleziono 50 linków na stronie 97.
Znaleziono 50 linków na stronie 98.
Znaleziono 50 linków na stronie 99.
Znaleziono 50 linków na stronie 100.


In [16]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book81_100.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book81_100.csv'")

Dane zostały zapisane do pliku 'book81_100.csv'


In [17]:
# Tworzenie przeglądarki tylko raz 6
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(101, 121):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 101.
Znaleziono 50 linków na stronie 102.
Znaleziono 50 linków na stronie 103.
Znaleziono 50 linków na stronie 104.
Znaleziono 50 linków na stronie 105.
Znaleziono 50 linków na stronie 106.
Znaleziono 50 linków na stronie 107.
Znaleziono 0 linków na stronie 108.
Znaleziono 50 linków na stronie 109.
Znaleziono 50 linków na stronie 110.
Znaleziono 50 linków na stronie 111.
Znaleziono 50 linków na stronie 112.
Znaleziono 50 linków na stronie 113.
Znaleziono 50 linków na stronie 114.
Znaleziono 50 linków na stronie 115.
Znaleziono 50 linków na stronie 116.
Znaleziono 50 linków na stronie 117.
Znaleziono 50 linków na stronie 118.
Znaleziono 50 linków na stronie 119.
Znaleziono 50 linków na stronie 120.


In [18]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book101_120.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book101_120.csv'")

Dane zostały zapisane do pliku 'book101_120.csv'


In [19]:
# Tworzenie przeglądarki tylko raz 7
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(121, 141):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 121.
Znaleziono 50 linków na stronie 122.
Znaleziono 50 linków na stronie 123.
Znaleziono 50 linków na stronie 124.
Znaleziono 50 linków na stronie 125.
Znaleziono 50 linków na stronie 126.
Znaleziono 50 linków na stronie 127.
Znaleziono 50 linków na stronie 128.
Znaleziono 50 linków na stronie 129.
Znaleziono 50 linków na stronie 130.
Znaleziono 50 linków na stronie 131.
Znaleziono 0 linków na stronie 132.
Znaleziono 50 linków na stronie 133.
Znaleziono 50 linków na stronie 134.
Znaleziono 0 linków na stronie 135.
Znaleziono 0 linków na stronie 136.
Znaleziono 0 linków na stronie 137.
Znaleziono 50 linków na stronie 138.
Znaleziono 0 linków na stronie 139.
Znaleziono 50 linków na stronie 140.


In [20]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book121_140.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book121_140.csv'")

Dane zostały zapisane do pliku 'book121_140.csv'


In [21]:
# Tworzenie przeglądarki tylko raz 8
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(141, 161):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 141.
Znaleziono 50 linków na stronie 142.
Znaleziono 50 linków na stronie 143.
Znaleziono 50 linków na stronie 144.
Znaleziono 0 linków na stronie 145.
Znaleziono 0 linków na stronie 146.
Znaleziono 50 linków na stronie 147.
Znaleziono 50 linków na stronie 148.
Znaleziono 50 linków na stronie 149.
Znaleziono 50 linków na stronie 150.
Znaleziono 50 linków na stronie 151.
Znaleziono 50 linków na stronie 152.
Znaleziono 50 linków na stronie 153.
Znaleziono 50 linków na stronie 154.
Znaleziono 50 linków na stronie 155.
Znaleziono 50 linków na stronie 156.
Znaleziono 50 linków na stronie 157.
Znaleziono 0 linków na stronie 158.
Znaleziono 50 linków na stronie 159.
Znaleziono 50 linków na stronie 160.


In [22]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book141_160.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book141_160.csv'")

Dane zostały zapisane do pliku 'book141_160.csv'


In [23]:
# Tworzenie przeglądarki tylko raz 9
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(161, 181):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 161.
Znaleziono 50 linków na stronie 162.
Znaleziono 50 linków na stronie 163.
Znaleziono 50 linków na stronie 164.
Znaleziono 50 linków na stronie 165.
Znaleziono 50 linków na stronie 166.
Znaleziono 50 linków na stronie 167.
Znaleziono 50 linków na stronie 168.
Znaleziono 50 linków na stronie 169.
Znaleziono 50 linków na stronie 170.
Znaleziono 50 linków na stronie 171.
Znaleziono 50 linków na stronie 172.
Znaleziono 50 linków na stronie 173.
Znaleziono 50 linków na stronie 174.
Znaleziono 50 linków na stronie 175.
Znaleziono 50 linków na stronie 176.
Znaleziono 0 linków na stronie 177.
Znaleziono 50 linków na stronie 178.
Znaleziono 50 linków na stronie 179.
Znaleziono 50 linków na stronie 180.


In [24]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book161_180.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book161_180.csv'")

Dane zostały zapisane do pliku 'book161_180.csv'


In [43]:
# Tworzenie przeglądarki tylko raz 10
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(181, 201):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 181.
Znaleziono 50 linków na stronie 182.
Znaleziono 50 linków na stronie 183.
Znaleziono 0 linków na stronie 184.
Znaleziono 0 linków na stronie 185.
Znaleziono 50 linków na stronie 186.
Znaleziono 0 linków na stronie 187.
Znaleziono 0 linków na stronie 188.
Znaleziono 50 linków na stronie 189.
Znaleziono 50 linków na stronie 190.
Znaleziono 50 linków na stronie 191.
Znaleziono 50 linków na stronie 192.
Znaleziono 50 linków na stronie 193.
Znaleziono 50 linków na stronie 194.
Znaleziono 50 linków na stronie 195.
Znaleziono 0 linków na stronie 196.
Znaleziono 50 linków na stronie 197.
Znaleziono 0 linków na stronie 198.
Znaleziono 0 linków na stronie 199.
Znaleziono 0 linków na stronie 200.


In [26]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book181_200.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book181_200.csv'")

Dane zostały zapisane do pliku 'book181_200.csv'


In [29]:
# Tworzenie przeglądarki tylko raz 11 (strona 22)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(22, 23):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 0 linków na stronie 22.


In [30]:
# Tworzenie przeglądarki tylko raz 12 (strona 25)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(25, 26):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 25.


In [31]:
# Tworzenie przeglądarki tylko raz 13 (strony 28,29,30)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(28, 31):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 28.
Znaleziono 0 linków na stronie 29.
Znaleziono 50 linków na stronie 30.


In [32]:
# Tworzenie przeglądarki tylko raz 15 (strona 32)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(32, 33):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 32.


In [33]:
# Tworzenie przeglądarki tylko raz 16 (strona 36,37,38)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(36, 39):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 36.
Znaleziono 50 linków na stronie 37.
Znaleziono 50 linków na stronie 38.


In [34]:
# Tworzenie przeglądarki tylko raz 17 (strona 40)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(40, 41):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 40.


In [62]:
# Tworzenie przeglądarki tylko raz 18 (strona 63,64,65)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(63, 66):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 63.
Znaleziono 50 linków na stronie 64.
Znaleziono 0 linków na stronie 65.


In [36]:
# Tworzenie przeglądarki tylko raz 19 (strona 68,69,70)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(68, 71):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 68.
Znaleziono 50 linków na stronie 69.
Znaleziono 50 linków na stronie 70.


In [37]:
# Tworzenie przeglądarki tylko raz 20 (strona 82,83)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(82, 84):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 82.
Znaleziono 0 linków na stronie 83.


In [38]:
# Tworzenie przeglądarki tylko raz 21 (strona 89)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(89, 90):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 89.


In [39]:
# Tworzenie przeglądarki tylko raz 22 (strona 108)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(108, 109):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 108.


In [40]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book_do108_ubytki.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book_do108_ubytki.csv'")

Dane zostały zapisane do pliku 'book_do108_ubytki.csv'


In [63]:
# Tworzenie przeglądarki tylko raz 23 (strona 132)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(132, 133):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 132.


In [45]:
# Tworzenie przeglądarki tylko raz 24 (strona 135,136,137)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(135, 138):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 135.
Znaleziono 50 linków na stronie 136.
Znaleziono 50 linków na stronie 137.


In [64]:
# Tworzenie przeglądarki tylko raz 25 (strona 139)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(139, 140):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 139.


In [47]:
# Tworzenie przeglądarki tylko raz 26 (strona 145,146)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(145, 147):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 145.
Znaleziono 50 linków na stronie 146.


In [65]:
# Tworzenie przeglądarki tylko raz 27 (strona 158)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(158, 159):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 158.


In [49]:
# Tworzenie przeglądarki tylko raz 28 (strona 177)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(177, 178):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 177.


In [66]:
# Tworzenie przeglądarki tylko raz 29 (strona 22)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(22, 23):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 22.


In [51]:
# Tworzenie przeglądarki tylko raz 30 (strona 29)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(29, 30):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 29.


In [52]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book_do29_ubytki3.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book_do29_ubytki3.csv'")

Dane zostały zapisane do pliku 'book_do29_ubytki3.csv'


In [54]:
# Tworzenie przeglądarki tylko raz 31 (strona 83)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(83, 84):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 83.


In [55]:
# Tworzenie przeglądarki tylko raz 32 (strona 10)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(10, 11):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 10.


In [56]:
# Tworzenie przeglądarki tylko raz 33 (strona 13)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(13, 14):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 13.


In [57]:
# Tworzenie przeglądarki tylko raz 33 (strona 19,20)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(19, 21):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 19.
Znaleziono 50 linków na stronie 20.


In [58]:
# Tworzenie przeglądarki tylko raz 35 (strona 42,43,44)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(42, 45):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 42.
Znaleziono 50 linków na stronie 43.
Znaleziono 50 linków na stronie 44.


In [67]:
# Tworzenie przeglądarki tylko raz 36 (strona 49)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(49, 50):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 49.


In [60]:
# Tworzenie przeglądarki tylko raz 37 (strona 59)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(59, 60):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 59.


In [61]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book_do59_ubytki4.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book_do59_ubytki4.csv'")

Dane zostały zapisane do pliku 'book_do59_ubytki4.csv'


In [68]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book_do49_ubytki5.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book_do49_ubytki5.csv'")

Dane zostały zapisane do pliku 'book_do49_ubytki5.csv'


In [74]:
# Tworzenie przeglądarki tylko raz 38 (strona 184,185)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(184, 186):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 184.
Znaleziono 0 linków na stronie 185.


In [70]:
# Tworzenie przeglądarki tylko raz 39 (strona 187,188)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(187, 189):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 187.
Znaleziono 50 linków na stronie 188.


In [71]:
# Tworzenie przeglądarki tylko raz 40 (strona 196)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(196, 197):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 196.


In [76]:
# Tworzenie przeglądarki tylko raz 41 (strona 198,199,200)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(198, 201):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 198.
Znaleziono 0 linków na stronie 199.
Znaleziono 0 linków na stronie 200.


In [73]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("book_do200_ubytki6.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'book_do200_ubytki6.csv'")

Dane zostały zapisane do pliku 'book_do200_ubytki6.csv'


In [75]:
# Tworzenie przeglądarki tylko raz 42 (strona 65)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(65, 66):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 65.


In [77]:
# Tworzenie przeglądarki tylko raz 43 (strona 185)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(185, 186):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 185.


In [81]:
# Tworzenie przeglądarki tylko raz 44 (strona 199, 200)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(199, 200):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 199.


In [82]:
# Tworzenie przeglądarki tylko raz 44 (strona 200)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # Przejście przez strony z wynikami wyszukiwania
    for page in range(200, 201):  # Możesz dostosować liczbę stron
        page_url = base_url.format(page)
        book_links = get_book_links(driver, page_url)
        print(f"Znaleziono {len(book_links)} linków na stronie {page}.")
        
        for book_url in book_links:
            book_data = scrape_book_details(driver, book_url, xpaths)
            all_books_data.append(book_data)

finally:
    driver.quit() 

Znaleziono 50 linków na stronie 200.


In [84]:
# Tworzenie DataFrame i zapis do CSV
df = pd.DataFrame(all_books_data)
df.to_csv("bookBeat_all2.csv", index=False, encoding="utf-8")

print("Dane zostały zapisane do pliku 'bookBeat_all2.csv'")

Dane zostały zapisane do pliku 'bookBeat_all2.csv'
